# LangGraph Persistent Memory Agent - Complete Implementation

This notebook demonstrates how to build an intelligent agent with persistent memory using LangGraph and FAISS.

## 📋 What We'll Build:
- An agent that remembers conversations across sessions
- Integration with web search and calculator tools
- Intelligent query routing
- Vector-based memory retrieval using FAISS

---

## 🔧 Step 1: Installation

Run this cell to install all required packages.

In [ ]:
# Install required packages
# Uncomment the line below if running for the first time
# !pip install -U langchain langgraph langchain-openai duckduckgo-search faiss-cpu python-dotenv

## 🔑 Step 2: Set Up API Keys

You need an OpenAI API key for this tutorial. Get one at: https://platform.openai.com/

In [ ]:
import os
from getpass import getpass

# Set your OpenAI API key
# Option 1: Set it directly (not recommended for production)
# os.environ["OPENAI_API_KEY"] = "sk-..."

# Option 2: Enter it securely when prompted
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

print("✅ API key configured")

## 📦 Step 3: Import Dependencies

Import all necessary libraries for building our agent.

In [ ]:
# LangChain components for OpenAI integration
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Vector store for persistent memory
from langchain_community.vectorstores import FAISS

# Tools for extending agent capabilities
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.tools import tool

# LangGraph for building the agent workflow
from langgraph.graph import StateGraph, END

# Standard library imports
import os
import re
from typing import Dict, Any

print("✅ All dependencies imported successfully")

## 🧠 Step 4: Initialize Persistent Vector Store

The vector store is the brain of our memory system. It stores conversation embeddings and enables similarity-based retrieval.

In [ ]:
# Define the path where our memory will be saved
# This file will persist across sessions
VECTOR_DB_PATH = "chat_memory.faiss"

# Initialize OpenAI embeddings model
# This converts text into 1536-dimensional vectors
# We use 'text-embedding-3-small' for a good balance of quality and speed
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Check if we have existing memory from previous sessions
if os.path.exists(VECTOR_DB_PATH):
    # Load existing memory from disk
    # allow_dangerous_deserialization=True is needed to load pickled data
    vectorstore = FAISS.load_local(
        VECTOR_DB_PATH, 
        embeddings, 
        allow_dangerous_deserialization=True
    )
    print("✅ Loaded existing memory from disk")
    print(f"📊 Current memory contains {vectorstore.index.ntotal} conversation entries")
else:
    # Create a new vector store with an initialization message
    vectorstore = FAISS.from_texts(
        ["Conversation initialized. This is the first session."], 
        embeddings
    )
    print("🆕 Created new memory store")

# The vectorstore is now ready to:
# 1. Store new conversation embeddings
# 2. Retrieve similar past conversations
# 3. Persist to disk for future sessions

## 🛠️ Step 5: Define Agent Tools

Tools extend our agent's capabilities beyond text generation. We'll create a calculator and web search tool.

In [ ]:
# Initialize the DuckDuckGo search tool
# This allows our agent to search the web for current information
search_tool = DuckDuckGoSearchRun()

# Create a custom calculator tool using the @tool decorator
@tool
def calculator(expression: str) -> str:
    """
    Evaluate a simple mathematical expression.
    
    Args:
        expression: A math expression like '2+2', '10*5', or '100/4'
    
    Returns:
        The result of the calculation or an error message
    
    Examples:
        calculator('2+2') -> 'The result is 4'
        calculator('10*5') -> 'The result is 50'
    """
    try:
        # Clean the expression to only allow safe mathematical operations
        # This prevents code injection attacks
        safe_expr = re.sub(r'[^0-9+\-*/.() ]', '', expression)
        
        # Evaluate the mathematical expression
        # Note: In production, consider using a safer math parser
        result = eval(safe_expr)
        
        return f"The result is {result}"
    except Exception as e:
        # Return a helpful error message if calculation fails
        return f"Error evaluating expression: {e}"

print("✅ Tools initialized successfully")
print("🔍 Available tools: Web Search, Calculator")

## 🤖 Step 6: Initialize the Language Model

We'll use GPT-4o-mini for generating responses and reasoning.

In [ ]:
# Initialize the ChatOpenAI model
# - model: gpt-4o-mini is fast and cost-effective
# - temperature: 0 makes responses more deterministic and focused
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

print("✅ Language model initialized")
print("🧠 Using model: gpt-4o-mini")

## 🎯 Step 7: Define Agent Nodes

Each node represents a specialized processing step in our agent workflow.

### 7.1 Controller Node (Query Router)

In [ ]:
def controller(state: Dict[str, Any]) -> Dict[str, str]:
    """
    Route incoming queries to the appropriate specialized node.
    
    This is the "brain" of our routing system. It analyzes the user's query
    and decides which specialized node should handle it.
    
    Args:
        state: Dictionary containing the user's query
    
    Returns:
        Dictionary with 'next' key indicating which node to route to
    
    Routing Logic:
        - If query contains math operations → route to 'calc'
        - If query asks for information → route to 'search'
        - Otherwise → route to 'reason' for general conversation
    """
    # Extract the query and convert to lowercase for easier matching
    query = state["query"].lower()
    
    # Check if the query contains mathematical operations or keywords
    math_keywords = ["+", "-", "*", "/", "multiply", "divide", "calculate", "sum", "add"]
    if any(keyword in query for keyword in math_keywords):
        print("🧮 Controller: Routing to Calculator node")
        return {"next": "calc"}
    
    # Check if the query is asking for information that requires search
    search_keywords = ["who", "what", "when", "where", "news", "latest", "search", "find", "recent"]
    if any(keyword in query for keyword in search_keywords):
        print("🔍 Controller: Routing to Search node")
        return {"next": "search"}
    
    # Default to reasoning node for conversational queries
    print("💭 Controller: Routing to Reasoning node")
    return {"next": "reason"}

print("✅ Controller node defined")

### 7.2 Reasoning Node (Memory-Enhanced Conversation)

In [ ]:
def reasoning(state: Dict[str, Any]) -> Dict[str, str]:
    """
    Handle conversational queries with memory-enhanced context.
    
    This is the most sophisticated node. It:
    1. Retrieves relevant past conversations from vector memory
    2. Includes this context in the prompt to the LLM
    3. Generates a contextually-aware response
    4. Saves the new conversation to memory for future reference
    
    Args:
        state: Dictionary containing the user's query
    
    Returns:
        Dictionary with 'answer' key containing the response
    """
    query = state["query"]
    print(f"💭 Reasoning: Processing query: '{query}'")
    
    # Step 1: Retrieve similar past conversations from memory
    # k=3 means we get the 3 most similar past exchanges
    print("   🔍 Searching memory for relevant context...")
    retrieved_docs = vectorstore.similarity_search(query, k=3)
    
    # Extract the text content from retrieved documents
    memory_context = "\n".join([doc.page_content for doc in retrieved_docs])
    print(f"   📚 Found {len(retrieved_docs)} relevant memories")
    
    # Step 2: Build a prompt that includes the memory context
    # This helps the LLM understand the conversation history
    prompt = f"""You are a helpful AI assistant with memory of past conversations.

Previous relevant context from our conversation history:
{memory_context}

Current user question: {query}

Instructions:
- Respond clearly and naturally
- If the previous context is relevant, acknowledge and build upon it
- If it's not relevant, answer the question directly
- Be conversational and helpful"""
    
    # Step 3: Generate response using the LLM
    print("   🤖 Generating response...")
    response = llm.invoke(prompt)
    
    # Step 4: Save this exchange to memory for future reference
    # Format: "User: [question]\nAssistant: [answer]"
    conversation_entry = f"User: {query}\nAssistant: {response.content}"
    vectorstore.add_texts([conversation_entry])
    
    # Step 5: Persist the updated memory to disk
    vectorstore.save_local(VECTOR_DB_PATH)
    print("   💾 Memory updated and saved")
    
    return {"answer": response.content}

print("✅ Reasoning node defined")

### 7.3 Calculator Node (Mathematical Operations)

In [ ]:
def calc(state: Dict[str, Any]) -> Dict[str, str]:
    """
    Handle mathematical calculations.
    
    This node:
    1. Extracts the mathematical expression from the query
    2. Uses the calculator tool to compute the result
    3. Saves the calculation to memory
    
    Args:
        state: Dictionary containing the user's query
    
    Returns:
        Dictionary with 'answer' key containing the calculation result
    """
    query = state["query"]
    print(f"🧮 Calculator: Processing calculation: '{query}'")
    
    # Extract only the mathematical expression from the query
    # This removes text like "what is" or "calculate"
    expr = re.sub(r"[^0-9+\-*/(). ]", "", query)
    print(f"   📊 Extracted expression: '{expr}'")
    
    # Use the calculator tool to compute the result
    result = calculator.invoke(expr)
    print(f"   ✅ Result: {result}")
    
    # Save this calculation to memory
    conversation_entry = f"User: {query}\nAssistant: {result}"
    vectorstore.add_texts([conversation_entry])
    vectorstore.save_local(VECTOR_DB_PATH)
    print("   💾 Calculation saved to memory")
    
    return {"answer": result}

print("✅ Calculator node defined")

### 7.4 Search Node (Web Information Retrieval)

In [ ]:
def search(state: Dict[str, Any]) -> Dict[str, str]:
    """
    Handle web search queries.
    
    This node:
    1. Performs a web search using DuckDuckGo
    2. Uses the LLM to summarize the search results
    3. Saves the search and summary to memory
    
    Args:
        state: Dictionary containing the user's query
    
    Returns:
        Dictionary with 'answer' key containing the summarized results
    """
    query = state["query"]
    print(f"🔍 Search: Searching for: '{query}'")
    
    try:
        # Perform the web search
        print("   🌐 Querying DuckDuckGo...")
        search_results = search_tool.run(query)
        print(f"   📄 Retrieved {len(search_results)} characters of results")
        
        # Use the LLM to create a concise summary of the results
        summary_prompt = f"""Summarize the following search results concisely and clearly:

{search_results}

Provide a clear, informative summary that answers the user's question.
Focus on the most relevant and important information."""
        
        print("   🤖 Generating summary...")
        summary = llm.invoke(summary_prompt)
        
        # Save the search query and summary to memory
        conversation_entry = f"User: {query}\nAssistant: {summary.content}"
        vectorstore.add_texts([conversation_entry])
        vectorstore.save_local(VECTOR_DB_PATH)
        print("   💾 Search results saved to memory")
        
        return {"answer": summary.content}
    
    except Exception as e:
        # Handle search errors gracefully
        error_msg = f"I encountered an error while searching: {str(e)}. Please try rephrasing your question."
        print(f"   ❌ Search error: {e}")
        return {"answer": error_msg}

print("✅ Search node defined")

## 🔗 Step 8: Build the LangGraph Workflow

Now we'll connect all the nodes into a state machine that orchestrates the entire agent workflow.

In [ ]:
# Initialize the StateGraph with a dictionary state type
# The state will store the query and answer as it flows through nodes
graph = StateGraph(dict)

# Add all our nodes to the graph
print("🔧 Building agent workflow...")
graph.add_node("controller", controller)  # Entry point that routes queries
graph.add_node("reason", reasoning)       # Handles conversational queries
graph.add_node("calc", calc)              # Handles mathematical calculations
graph.add_node("search", search)          # Handles web searches
print("   ✅ Added 4 nodes: controller, reason, calc, search")

# Set the controller as the entry point
# Every query will start here for routing
graph.set_entry_point("controller")
print("   ✅ Set controller as entry point")

# Add conditional edges from the controller
# Based on the controller's decision, route to the appropriate node
graph.add_conditional_edges(
    "controller",                    # Source node
    lambda x: x["next"],             # Function that returns the next node name
    {
        "reason": "reason",          # If next="reason", go to reason node
        "calc": "calc",              # If next="calc", go to calc node
        "search": "search"           # If next="search", go to search node
    }
)
print("   ✅ Added conditional routing from controller")

# All specialized nodes lead to END
# Once any node completes, the workflow ends
graph.add_edge("reason", END)
graph.add_edge("calc", END)
graph.add_edge("search", END)
print("   ✅ Connected all nodes to END")

# Compile the graph into a runnable application
app = graph.compile()
print("\n✅ Agent workflow compiled successfully!")
print("\n📊 Workflow structure:")
print("   START → controller → [reason|calc|search] → END")

## 🎨 Optional: Visualize the Workflow

Let's visualize our agent's workflow graph (requires additional packages).

In [ ]:
# Optional: Visualize the graph
# Uncomment if you have graphviz installed
# from IPython.display import Image, display
# try:
#     display(Image(app.get_graph().draw_mermaid_png()))
# except Exception as e:
#     print(f"Could not visualize graph: {e}")
#     print("Install graphviz to see the visualization")

print("ℹ️  To visualize the workflow graph, install graphviz and uncomment the code above")

## 🧪 Step 9: Test Individual Components

Before running the full chat interface, let's test each component.

In [ ]:
# Test the calculator tool
print("🧪 Testing Calculator:")
test_calc_result = calculator.invoke("25 * 4")
print(f"   25 * 4 = {test_calc_result}")

# Test the agent with a math query
print("\n🧪 Testing Full Agent with Math Query:")
result = app.invoke({"query": "What is 144 divided by 12?"})
print(f"   Answer: {result['answer']}")

print("\n✅ Component tests passed!")

## 💬 Step 10: Interactive Chat Interface

Now let's create an interactive chat function to talk with our agent.

In [ ]:
def chat():
    """
    Run an interactive chat session with the persistent memory agent.
    
    Features:
    - Accepts user input in a loop
    - Processes queries through the agent workflow
    - Displays responses with formatting
    - Saves all conversations to memory
    - Exits gracefully on 'exit' or 'quit' commands
    """
    print("\n" + "="*60)
    print("🤖 PERSISTENT MEMORY AGENT")
    print("="*60)
    print("\n💾 Memory is automatically saved after each message")
    print("🔄 Your conversations will persist across sessions")
    print("\nType 'exit', 'quit', or 'bye' to end the conversation")
    print("Type 'memory' to see memory stats")
    print("\n" + "-"*60 + "\n")
    
    while True:
        # Get user input
        query = input("😊 You: ").strip()
        
        # Check for exit commands
        if query.lower() in ["exit", "quit", "bye"]:
            print("\n" + "-"*60)
            print("🧠 All conversations saved to memory")
            print("👋 Goodbye! Your agent will remember this conversation.")
            print("="*60 + "\n")
            break
        
        # Check for memory stats command
        if query.lower() == "memory":
            print(f"\n📊 Memory Statistics:")
            print(f"   Total entries: {vectorstore.index.ntotal}")
            print(f"   Storage file: {VECTOR_DB_PATH}")
            print()
            continue
        
        # Skip empty inputs
        if not query:
            continue
        
        print()  # Add spacing
        
        # Run the agent
        try:
            result = app.invoke({"query": query})
            print(f"\n🤖 AI: {result['answer']}\n")
            print("-"*60 + "\n")
        except Exception as e:
            print(f"\n❌ Error: {e}")
            print("Please try again with a different query.\n")

print("✅ Chat function defined")
print("\nRun the cell below to start chatting!")

## 🚀 Step 11: Start Chatting!

Run this cell to start an interactive conversation with your agent.

In [ ]:
# Start the interactive chat session
chat()

## 🧪 Step 12: Example Queries to Try

Here are some example queries to test different capabilities:

In [ ]:
# Test queries programmatically (without interactive mode)

test_queries = [
    "Who is the current president of the United States?",
    "What is 25 multiplied by 8?",
    "Tell me about recent developments in AI",
    "What did we just discuss?",  # Tests memory
]

print("🧪 Running test queries:\n")
print("="*60 + "\n")

for i, query in enumerate(test_queries, 1):
    print(f"Query {i}: {query}")
    result = app.invoke({"query": query})
    print(f"Answer: {result['answer']}")
    print("\n" + "-"*60 + "\n")

## 🔍 Step 13: Inspect Memory Contents

Let's look at what's stored in our vector memory.

In [ ]:
# Display memory statistics
print("📊 Memory Statistics:")
print(f"   Total conversation entries: {vectorstore.index.ntotal}")
print(f"   Storage file: {VECTOR_DB_PATH}")
print(f"   File exists: {os.path.exists(VECTOR_DB_PATH)}")

# Search memory for a specific topic
print("\n🔍 Sample Memory Search:")
sample_query = "What have we discussed?"
results = vectorstore.similarity_search(sample_query, k=5)
print(f"   Query: '{sample_query}'")
print(f"   Found {len(results)} relevant memories:\n")

for i, doc in enumerate(results, 1):
    print(f"   {i}. {doc.page_content[:100]}...")
    print()

## 🧹 Step 14: Memory Management Functions

Utility functions for managing the vector memory.

In [ ]:
def clear_memory():
    """
    Clear all stored memories and start fresh.
    
    WARNING: This will delete all conversation history!
    """
    global vectorstore
    
    # Delete the FAISS files if they exist
    if os.path.exists(VECTOR_DB_PATH):
        os.remove(VECTOR_DB_PATH)
    if os.path.exists(f"{VECTOR_DB_PATH}.pkl"):
        os.remove(f"{VECTOR_DB_PATH}.pkl")
    
    # Create a fresh vector store
    vectorstore = FAISS.from_texts(
        ["Memory cleared. Starting fresh."], 
        embeddings
    )
    
    print("🧹 Memory cleared successfully")
    print("✨ Starting with a fresh memory store")


def export_memory_to_text(filename="memory_export.txt"):
    """
    Export all memories to a readable text file.
    
    Args:
        filename: Name of the output file
    """
    # Retrieve all memories (get a large k value)
    all_memories = vectorstore.similarity_search("", k=1000)
    
    # Write to file
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("AGENT MEMORY EXPORT\n")
        f.write("=" * 60 + "\n\n")
        
        for i, doc in enumerate(all_memories, 1):
            f.write(f"Entry {i}:\n")
            f.write(doc.page_content)
            f.write("\n" + "-" * 60 + "\n\n")
    
    print(f"📄 Memory exported to {filename}")
    print(f"   Total entries: {len(all_memories)}")


print("✅ Memory management functions defined")
print("\nAvailable functions:")
print("   - clear_memory(): Delete all stored memories")
print("   - export_memory_to_text(): Export memories to a text file")

## 📤 Step 15: Export Your Memories (Optional)

Export all stored conversations to a text file for review.

In [ ]:
# Uncomment to export memories
# export_memory_to_text("my_agent_memories.txt")

## 🎓 Summary and Key Concepts

### What We Built

1. **Persistent Vector Memory**: Using FAISS to store and retrieve conversation embeddings
2. **Intelligent Routing**: Controller node that directs queries to specialized handlers
3. **Multi-Tool Integration**: Calculator and web search capabilities
4. **Context-Aware Responses**: LLM uses retrieved memories for better answers
5. **Cross-Session Persistence**: Conversations are saved and loaded between sessions

### How Memory Works

```
User Query → Embedding (1536d vector) → Similarity Search → Top K Memories → Context
                                                                                  ↓
                                                                        LLM with Context
                                                                                  ↓
Response ← Save to Memory ← Extract Answer ← Generate Response
```

### Next Steps

1. **Add More Tools**: Email, calendar, database queries
2. **Multi-User Support**: Separate memory stores per user
3. **Memory Summarization**: Periodically compress old conversations
4. **Better Routing**: Use an LLM-based router instead of keyword matching
5. **Web Interface**: Deploy with Streamlit or FastAPI

---

**Congratulations! 🎉** You've built a sophisticated agent with persistent memory using LangGraph and FAISS.
